In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\SAIDT.dll"
#r ".\binaries\ApplicationWithIDT.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using ApplicationWithIDT;
using SAIDT;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


# Run Simulation

In [2]:
string dbPath = @"SAIDT_db" ;
var database = OpenOrCreateDatabase(dbPath);

In [3]:
Dictionary<string,int> DOFS = new Dictionary<string,int>();


## SinglePhaseField

In [4]:
using SAIDT;
BoSSS.Solution.Application.InitMPI();
BoSSS.Solution.Application.DeleteOldPlotFiles();
var pSAIDT = new SAIDTMain();
var CSAIDT = SAIDTHardCodedControl.StraightShock(
                    dbPath: dbPath,
                    MaxIterations: 50,
                    dgDegree: 0,
                    numOfCellsX: 5,
                    numOfCellsY: 5,
                    OptiNumOfCellsX: 5,
                    OptiNumOfCellsY: 5,
                    agg: 0.4,
                    ImmediatePlotPeriod: -1,
                    optiLevelSetType: OptiLevelSetType.SinglePhaseField,
                    LSDegree: 3
                    );
CSAIDT.SessionName="SAIDT_DGLevelSet";
CSAIDT.ProjectName="SAIDT_DGLevelSet";
pSAIDT.Init(CSAIDT);
pSAIDT.RunSolverMode();
DOFS.Add("SinglePhaseField",pSAIDT.LsTBO.DOFLocal);

In [5]:
pSAIDT.ResNorms.SaveToTextFile(pSAIDT.Control.ProjectName+"_ResPlot.txt");
pSAIDT.obj_f_vals.SaveToTextFile(pSAIDT.Control.ProjectName+"_EnResPlot.txt");
pSAIDT.GetResPlot().ToGnuplot().PlotSVG(xRes:600,yRes:500)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||R(z)|| 
 
 
 ||R(z)|| 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r(z)|| 
 
 
 ||r(z)||

In [6]:
var si= database.Sessions.Pick(0);
BoSSS.Solution.Application.DeleteOldPlotFiles();
pSAIDT.PlotShadowFields(si);


rm;


## Spline level set

In [7]:
BoSSS.Solution.Application.InitMPI();
var pSAIDT = new SAIDTMain();
var CSAIDT = SAIDTHardCodedControl.StraightShock(
                    dbPath: dbPath,
                    MaxIterations: 100,
                    dgDegree: 0,
                    numOfCellsX: 5,
                    numOfCellsY: 5,
                    OptiNumOfCellsX: 5,
                    OptiNumOfCellsY: 5,
                    agg: 0.4,
                    ImmediatePlotPeriod: -1,
                    optiLevelSetType: OptiLevelSetType.SplineLevelSet,
                    LSDegree: 3,
                    isFarConfig:true
                    );
CSAIDT.SessionName="SAIDT_SplineLevelSet";
CSAIDT.ProjectName="SAIDT_SplineLevelSet";
pSAIDT.Init(CSAIDT);
pSAIDT.RunSolverMode();
DOFS.Add("SplineLevelSet",pSAIDT.LevelSetOpti.GetLength());


Session ID: 2342a63f-ba56-48fe-bd51-f1babfd37f44, DB path: 'SAIDT_db'
Session directory 'SAIDT_db\sessions\2342a63f-ba56-48fe-bd51-f1babfd37f44'.
Grid repartitioning method: METIS
Grid repartitioning options: 
Number of cell Weights: 0


*****************************************************************************************
*  X   X   DDD   GGG   -   III  SSS  TTTTT  *
*   X X    D  D G      -    I  S      T     *
*    X     D  D G  GG  -    I   SSS   T     *      XDG Implicit Shock Tracking Solver    
*   X X    D  D G   G  -    I      S  T     *
*  X   X   DDD   GGG   -   III  SSS   T     *
*****************************************************************************************
*  
*  solver:                             SAIDT.SAIDTMain
*  project:                            SAIDT_SplineLevelSet
*  session:                            SAIDT_SplineLevelSet
*  optimization problem:               FullEnRes
*  merit function:                     L1Merit
*  level set type:               

In [8]:
pSAIDT.ResNorms.SaveToTextFile(pSAIDT.Control.ProjectName+"_ResPlot.txt");
pSAIDT.obj_f_vals.SaveToTextFile(pSAIDT.Control.ProjectName+"_EnResPlot.txt");
pSAIDT.GetResPlot().ToGnuplot().PlotSVG(xRes:600,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -12 
 
 
 
 
 10 -10 
 
 
 
 
 10 -8 
 
 
 
 
 10 -6 
 
 
 
 
 10 -4 
 
 
 
 
 10 -2 
 
 
 
 
 10 0 
 
 
 
 
 10 2 
 
 
 
 
 0 
 
 
 
 
 10 
 
 
 
 
 20 
 
 
 
 
 30 
 
 
 
 
 40 
 
 
 
 
 50 
 
 
 
 
 60 
 
 
 
 
 70 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||R(z)|| 
 
 
 ||R(z)|| 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M388.5,57.1 L441.9,57.1 M53.9,88.9 L61.0,91.6 L68.1,93.4 L75.2,94.6 L82.3,94.7 L89.3,96.5
 L96.4,96.8 L103.5,96.8 L110.6,96.9 L117.7,96.9 L124.8,96.9 L131.9,96.9 L139.0,96.9 L146.1,96.9
 L153.1,96.9 L160.2,96.9 L167.3,100.9 L174.4,102.8 L181.5,103.4 L188.6,103.5 L195.7,103.6 L202.8,103.6
 L209.8,103.6 L216.9,103.6 L224.0,103.6 L231.1,103.6 L238.2,103.6 L245.3,123.6 L252.4,182.9 L259.5,229.2
 L266.6,278.4 L273.6,317.0 L280.7,352.6 L287.8,357.3 L294.9,358.8 L302.0,365.1 L309.1,364.6 L316.2,367.0
 L323.3,367.1 L330.4,368.0 L337.4,368.0 L344.5,368.9 L351.6,368.9 L358.7,369.9 L365.8,369.8 L372.9,370.7
 L380.0,370.7 L387.1,371.6 L394.2,371.5 L401.2,372.4 L408.3,372.4 L415.4,373.3 L422.5,373.2 L429.6,374.0
 L436.7,374.0 L443.8,374.8 L450.9,374.8 L457.9,375.5 L465.0,376.6 L472.1,376.4 L479.2,377.1 L486.3,377.1
 L493.4,377.1 L500.5,377.1 L507.6,377.1 L514.7,377.1 L521.7,377.1 L528.8,377.1 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r(z)|| 
 
 
 ||r(z)|| 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M388.5,81.1 L441.9,81.1 M53.9,113.1 L61.0,118.7 L68.1,123.3 L75.2,115.3 L82.3,121.7 L89.3,125.4
 L96.4,126.2 L103.5,126.4 L110.6,126.5 L117.7,126.5 L124.8,126.5 L131.9,126.5 L139.0,126.5 L146.1,126.5
 L153.1,126.5 L160.2,126.5 L167.3,124.4 L174.4,127.8 L181.5,127.3 L188.6,127.5 L195.7,127.6 L202.8,127.6
 L209.8,127.6 L216.9,127.6 L224.0,127.6 L231.1,127.6 L238.2,127.6 L245.3,138.2 L252.4,195.2 L259.5,242.0
 L266.6,291.4 L273.6,325.9 L280.7,363.3 L287.8,367.5 L294.9,371.9 L302.0,381.8 L309.1,382.0 L316.2,383.8
 L323.3,385.4 L330.4,384.9 L337.4,386.3 L344.5,385.8 L351.6,387.1 L358.7,386.6 L365.8,387.9 L372.9,387.4
 L380.0,388.7 L387.1,388.2 L394.2,389.4 L401.2,389.2 L408.3,390.2 L415.4,390.3 L422.5,390.9 L429.6,391.6
 L436.7,391.7 L443.8,393.0 L450.9,392.4 L457.9,394.5 L465.0,392.5 L472.1,394.3 L479.2,394.9 L486.3,394.9
 L493.4,394.9 L500.5,394.9 L507.6,394.9 L514.7,394.9 L521.7,394.9 L528.8,394.9 '/>

In [9]:
var si= database.Sessions.Pick(0);
pSAIDT.PlotShadowFields(si);

## SpecFemField

In [10]:
using SAIDT;
BoSSS.Solution.Application.InitMPI();
var pSAIDT = new SAIDTMain();
var CSAIDT = SAIDTHardCodedControl.StraightShock(
                    dbPath: dbPath,
                    MaxIterations: 50,
                    dgDegree: 0,
                    numOfCellsX: 5,
                    numOfCellsY: 5,
                    OptiNumOfCellsX: 5,
                    OptiNumOfCellsY: 5,
                    agg: 0.4,
                    ImmediatePlotPeriod: 5,
                    optiLevelSetType: OptiLevelSetType.SpecFemField,
                    LSDegree: 3,
                    isFarConfig:true
                    );
CSAIDT.SessionName="SAIDT_CGLevelSet";
CSAIDT.ProjectName="SAIDT_CGLevelSet";
pSAIDT.Init(CSAIDT);
pSAIDT.RunSolverMode();
DOFS.Add("SpecFemField",pSAIDT.LevelSetOpti.GetLength());

Session ID: 381fe707-6e64-43ad-9f27-b0b76775f612, DB path: 'SAIDT_db'
Session directory 'SAIDT_db\sessions\381fe707-6e64-43ad-9f27-b0b76775f612'.
Grid repartitioning method: METIS
Grid repartitioning options: 
Number of cell Weights: 0


*****************************************************************************************
*  X   X   DDD   GGG   -   III  SSS  TTTTT  *
*   X X    D  D G      -    I  S      T     *
*    X     D  D G  GG  -    I   SSS   T     *      XDG Implicit Shock Tracking Solver    
*   X X    D  D G   G  -    I      S  T     *
*  X   X   DDD   GGG   -   III  SSS   T     *
*****************************************************************************************
*  
*  solver:                             SAIDT.SAIDTMain
*  project:                            SAIDT_CGLevelSet
*  session:                            SAIDT_CGLevelSet
*  optimization problem:               FullEnRes
*  merit function:                     L1Merit
*  level set type:                     Sp

In [11]:
pSAIDT.ResNorms.SaveToTextFile(pSAIDT.Control.ProjectName+"_ResPlot.txt");
pSAIDT.obj_f_vals.SaveToTextFile(pSAIDT.Control.ProjectName+"_EnResPlot.txt");
pSAIDT.GetResPlot().ToGnuplot().PlotSVG(xRes:600,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -16 
 
 
 
 
 10 -14 
 
 
 
 
 10 -12 
 
 
 
 
 10 -10 
 
 
 
 
 10 -8 
 
 
 
 
 10 -6 
 
 
 
 
 10 -4 
 
 
 
 
 10 -2 
 
 
 
 
 10 0 
 
 
 
 
 10 2 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 30 
 
 
 
 
 35 
 
 
 
 
 40 
 
 
 
 
 45 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||R(z)|| 
 
 
 ||R(z)|| 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M388.5,57.1 L441.9,57.1 M53.9,77.8 L64.9,78.6 L76.0,80.1 L87.0,78.9 L98.0,80.1 L109.0,78.9
 L120.1,79.1 L131.1,80.7 L142.1,80.8 L153.1,80.8 L164.2,80.8 L175.2,81.6 L186.2,81.7 L197.2,81.8
 L208.3,82.2 L219.3,82.3 L230.3,82.4 L241.4,82.7 L252.4,82.7 L263.4,82.7 L274.4,82.7 L285.5,82.7
 L296.5,82.7 L307.5,82.7 L318.5,83.8 L329.6,81.2 L340.6,82.8 L351.6,84.8 L362.6,83.4 L373.7,86.3
 L384.7,87.4 L395.7,88.0 L406.8,88.3 L417.8,88.4 L428.8,88.5 L439.8,88.6 L450.9,88.7 L461.9,88.7
 L472.9,88.8 L483.9,88.8 L495.0,88.9 L506.0,88.9 L517.0,89.0 L528.0,89.0 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r(z)|| 
 
 
 ||r(z)|| 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M388.5,81.1 L441.9,81.1 M53.9,422.0 L64.9,106.7 L76.0,108.8 L87.0,102.2 L98.0,100.6 L109.0,98.7
 L120.1,98.3 L131.1,110.8 L142.1,111.2 L153.1,111.2 L164.2,111.2 L175.2,108.1 L186.2,112.7 L197.2,112.6
 L208.3,107.6 L219.3,107.9 L230.3,107.8 L241.4,107.3 L252.4,107.3 L263.4,107.3 L274.4,107.3 L285.5,107.3
 L296.5,107.3 L307.5,107.3 L318.5,103.0 L329.6,99.7 L340.6,97.6 L351.6,97.9 L362.6,97.0 L373.7,96.4
 L384.7,104.9 L395.7,103.4 L406.8,105.1 L417.8,105.1 L428.8,105.7 L439.8,106.3 L450.9,106.2 L461.9,106.4
 L472.9,106.3 L483.9,106.4 L495.0,106.5 L506.0,106.7 L517.0,106.7 L528.0,106.8 '/>

In [12]:
var si= database.Sessions.Pick(0);
pSAIDT.PlotShadowFields(si);

In [13]:
using System.IO;
static void AppendRowToCSV(string filePath, double[] rowData)
    {
        // Create a StreamWriter in append mode to append rows to the CSV file
        using (StreamWriter writer = new StreamWriter(filePath, true))
        {
            int cols = rowData.Length;

            // Write double values of the new row to the file
            for (int i = 0; i < cols; i++)
            {
                writer.Write(rowData[i]);

                // Add a comma after each value except for the last one in a row
                if (i < cols - 1)
                {
                    writer.Write(";");
                }
            }

            // Move to the next line after writing the row
            writer.WriteLine();
        }
    }
    static void WriteCSV(string filePath, string[] data)
    {
        // Create a StreamWriter to write to the CSV file
        using (StreamWriter writer = new StreamWriter(filePath))
        {
            int cols = data.Length;

            // Loop through the data array and write to the file
                for (int j = 0; j < cols; j++)
                {
                    // Write each cell value to the file
                    writer.Write(data[ j]);

                    // Add a comma after each cell value except for the last one in a row
                    if (j < cols - 1)
                    {
                        writer.Write(";");
                    }
                }

                // Move to the next line after each row
                writer.WriteLine();
            
        }
    }
    static void DeleteCSVFiles()
    {
        try
        {
            // Get all CSV files in the specified directory
            string[] csvFiles = Directory.GetFiles("*.csv");

            // Delete each CSV file
            foreach (string file in csvFiles)
            {
                File.Delete(file);
                Console.WriteLine($"Deleted: {file}");
            }
        }
        catch (Exception ex)
        {
            Console.WriteLine($"Error deleting CSV files: {ex.Message}");
        }
    }

## Measure wall-clock time for computation of sensitivities and residuals 

In [14]:
using System.Diagnostics;
BoSSS.Solution.Application.InitMPI();
DeleteCSVFiles();

int nRefs=4;
var optLsTypes=new OptiLevelSetType[] {OptiLevelSetType.SinglePhaseField,OptiLevelSetType.SplineLevelSet,OptiLevelSetType.SpecFemField};
var CellsVsRuntime_dRdPhi = MultidimensionalArray.Create(nRefs,optLsTypes.Length +1);
var CellsVsRuntime_Residuals = MultidimensionalArray.Create(nRefs,optLsTypes.Length +1);
var CellsVsRuntime_dRdU = MultidimensionalArray.Create(nRefs,optLsTypes.Length +1);
string[] stringArray = new string[] { "nCells" }.Concat(optLsTypes.Select(x => x.ToString())).ToArray();

//Create files for results
string filePathdRdPhi = "TimeDRdPhi.csv";
string filePathdRdU = "TimedRdU.csv";
string filePathResiduals = "TimeResiduals.csv";
string filePathDOFs = "DOFs.csv";
WriteCSV(filePathdRdPhi,stringArray);
WriteCSV(filePathdRdU,stringArray);
WriteCSV(filePathResiduals,stringArray);
WriteCSV(filePathDOFs,stringArray);

for(int n =0; n< nRefs;n++){
    //counter for level set type
    int iLS=1;
    //save cell resolution
    int reso= (int) Math.Pow(2,n+1);
    CellsVsRuntime_dRdPhi[n,0]=reso*reso;
    double[] dRdPhiResults= new double[optLsTypes.Length +1];
    dRdPhiResults[0]=reso*reso;
    double[] dRdUResults=dRdPhiResults.CloneAs();
    double[] resiudalResults=dRdPhiResults.CloneAs();
    double[] DOFs=dRdPhiResults.CloneAs();
    foreach(OptiLevelSetType lstype in optLsTypes){
        var pSAIDT = new SAIDTMain();
        var CSAIDT = SAIDTHardCodedControl.StraightShock(
                            dbPath: null,
                            MaxIterations: 1,
                            dgDegree: 0,
                            numOfCellsX: reso,
                            numOfCellsY: reso,
                            OptiNumOfCellsX: reso,
                            OptiNumOfCellsY: reso,
                            agg: 0.4,
                            ImmediatePlotPeriod: -1,
                            optiLevelSetType: lstype,
                            LSDegree: 3,
                            isFarConfig:true);

        pSAIDT.Init(CSAIDT);    
        pSAIDT.InitializeEverything();
        //pSAIDT.RunSolverMode();

        //Measure time for creation of Jacobians dRdPhi
        Stopwatch stopwatch = new Stopwatch();stopwatch.Start();
        pSAIDT.FD_LevelSet();
        stopwatch.Stop();
        CellsVsRuntime_dRdPhi[n,iLS] =stopwatch.ElapsedMilliseconds;
        dRdPhiResults[iLS]=stopwatch.ElapsedMilliseconds;

        //Measure time for creation of Jacobians dRdU
        stopwatch = new Stopwatch();stopwatch.Start();
        pSAIDT.Oproblem.GetJacobians(pSAIDT.ConservativeFields, pSAIDT.XSpatialOperator.LinearizationHint);
        stopwatch.Stop(); 
        CellsVsRuntime_dRdU[n,iLS] =stopwatch.ElapsedMilliseconds;
        dRdUResults[iLS]=stopwatch.ElapsedMilliseconds;

        //Measure time for evaluation of the Residuals
        stopwatch = new Stopwatch();stopwatch.Start();
        pSAIDT.ComputeResiduals();
        stopwatch.Stop(); Console.WriteLine("Elapsed Time: " + stopwatch.ElapsedMilliseconds + "ms");
        CellsVsRuntime_Residuals[n,iLS] =stopwatch.ElapsedMilliseconds;
        resiudalResults[iLS]=stopwatch.ElapsedMilliseconds;

        DOFs[iLS]=pSAIDT.LevelSetOpti.GetLength();

        iLS++;
    }
    // Append the new row to the existing CSV file
    AppendRowToCSV(filePathdRdPhi, dRdPhiResults);
    AppendRowToCSV(filePathdRdU, dRdUResults);
    AppendRowToCSV(filePathResiduals, resiudalResults);
    AppendRowToCSV(filePathDOFs, DOFs);
}

Error deleting CSV files: The filename, directory name, or volume label syntax is incorrect. : 'c:\experimental\public\examples\ShockFitting\Studies\ComparisonLevelSets\*.csv'
Session ID: 00000000-0000-0000-0000-000000000000, DB path: 'EMPTY'
Session directory 'EMPTY\sessions\00000000-0000-0000-0000-000000000000'.
Grid repartitioning method: METIS
Grid repartitioning options: 
Number of cell Weights: 0


*****************************************************************************************
*  X   X   DDD   GGG   -   III  SSS  TTTTT  *
*   X X    D  D G      -    I  S      T     *
*    X     D  D G  GG  -    I   SSS   T     *      XDG Implicit Shock Tracking Solver    
*   X X    D  D G   G  -    I      S  T     *
*  X   X   DDD   GGG   -   III  SSS   T     *
*****************************************************************************************
*  
*  solver:                             SAIDT.SAIDTMain
*  project:                            ScalarAdvection
*  session:             

In [15]:
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

In [16]:
CellsVsRuntime_dRdPhi

Dimension,2
Lengths,"[ 4, 4 ]"
Storage,"[ 4, 1158, 218, 683, 16, 5399, 359, 3087, 64, 29897, 905, 18592, 256, 239238, 3686, 166903 ]"
IsContinuous,True
StructureType,General
Length,16
NoOfCols,4
NoOfRows,4
IsLocked,False


In [17]:
var plot = new Plot2Ddata();
int count=0;
for(int iLS=0;iLS<CellsVsRuntime_dRdPhi.Lengths[1]-1; iLS ++){
    plot.AddDataGroup(optLsTypes[iLS].ToString(),CellsVsRuntime_dRdPhi.ExtractSubArrayShallow(-1,0).To1DArray(),CellsVsRuntime_dRdPhi.ExtractSubArrayShallow(-1,iLS+1).To1DArray(), GetFormat(count));
    count++;
}
plot.Xlabel="number of Cells";
plot.LogY = true;
plot.LogX = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 10 4 
 
 
 
 
 10 5 
 
 
 
 
 10 6 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 number of Cells 
 
 
 
 
 SinglePhaseField 
 
 
 SinglePhaseField 
 
 
 
 
 
 
 
 
 
 
 SplineLevelSet 
 
 
 SplineLevelSet 
 
 
 
 
 
 
 
 
 
 
 SpecFemField 
 
 
 SpecFemField